In [ ]:
!pip install -q cassandra-driver
!pip install -q cassio>=0.1.1
!pip install -q llama-index==0.10.17
%pip install llama-index-llms-langchain==0.1.2
%pip install llama-index-embeddings-langchain==0.1.1
%pip install llama-index-embeddings-voyageai==0.1.2
%pip install llama-index-llms-ollama==0.1.2
%pip install llama-index-embeddings-ollama==0.1.2
%pip install llama-index-vector-stores-cassandra==0.1.2
%pip install llama-index-llms-openrouter==0.1.2
!pip install -q tiktoken==0.4.0
!pip install --quiet "astrapy>=0.5.8"
!pip install langchain
!pip install voyageai
!pip install fastapi nest-asyncio pyngrok uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install gradientai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 13.9 MB/s eta 0:00:00


In [ ]:
import gradientai
from gradientai import Gradient

In [ ]:
gradient = Gradient(workspace_id='$$$$$',access_token='$$$$$')

In [ ]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import pandas as pd

from llama_index.llms.ollama import Ollama

from llama_index.core import Settings, StorageContext , set_global_service_context,get_response_synthesizer
from llama_index.core.indices.service_context import ServiceContext
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.chat_engine import CondensePlusContextChatEngine,SimpleChatEngine,ContextChatEngine
from llama_index.core.query_engine import BaseQueryEngine
from llama_index.core.schema import Document
from llama_index.core import VectorStoreIndex

from llama_index.embeddings.langchain import LangchainEmbedding
from langchain_community.embeddings import VoyageEmbeddings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.openrouter import OpenRouter
import voyageai

from IPython.display import Markdown, display

from llama_index.core import PromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.memory import ChatMemoryBuffer

import cassandra
import cassio
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
from llama_index.vector_stores.cassandra import CassandraVectorStore

print (cassandra.__version__)

3.29.1


In [ ]:
llama_debug = LlamaDebugHandler(print_trace_on_end=True,)
callback_manager = CallbackManager([llama_debug])

In [ ]:
lc_embed = VoyageEmbeddings(model="voyage-large-2",voyage_api_key="$$$$$")
embed_model = LangchainEmbedding(lc_embed,callback_manager=callback_manager)
llm = Ollama(model="aish/svce-ai", request_timeout=60.0,base_url="https://2e9c-34-16-170-111.ngrok-free.app",temperature=0.2,context_window=8192,callback_manager=callback_manager,max_new_tokens=6072)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `VoyageEmbeddings` was deprecated in LangChain 0.0.29 and will be removed in 0.3. An updated version of the class exists in the langchain-voyageai package and should be used instead. To use it run `pip install -U langchain-voyageai` and import as `from langchain_voyageai import VoyageAIEmbeddings`.
  warn_deprecated(


In [ ]:
Settings.embed_model = embed_model
Settings.llm = llm
Settings.chunk_size = 3072
Settings.callback_manager = callback_manager

# Global state mgmt
service_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model = embed_model,
    chunk_size=3072,
    callback_manager=callback_manager
)
set_global_service_context(service_context)

<ipython-input-9-91aff2feb32a>:5: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [ ]:
cassio.init(
    database_id="$$$$$",
    token="$$$$$",
    keyspace="default_keyspace",
)
cassandra_store = CassandraVectorStore(
    table="vo_l_2_1", embedding_dimension=1536,
)
storage_context = StorageContext.from_defaults(vector_store=cassandra_store)

Downgrading core protocol version from 66 to 65 for aac415f8-3179-4fd1-bed8-a21edcc9e9ce-us-east-2.db.astra.datastax.com:29042:75b303bc-3b8d-4f1c-887c-0b1c312baa70. To avoid this, it is best practice to explicitly set Cluster(protocol_version) to the version supported by your cluster. http://datastax.github.io/python-driver/api/cassandra/cluster.html#cassandra.cluster.Cluster.protocol_version


Downgrading core protocol version from 65 to 5 for aac415f8-3179-4fd1-bed8-a21edcc9e9ce-us-east-2.db.astra.datastax.com:29042:75b303bc-3b8d-4f1c-887c-0b1c312baa70. To avoid this, it is best practice to explicitly set Cluster(protocol_version) to the version supported by your cluster. http://datastax.github.io/python-driver/api/cassandra/cluster.html#cassandra.cluster.Cluster.protocol_version


ERROR:cassandra.connection:Closing connection <LibevConnection(139793671425568) aac415f8-3179-4fd1-bed8-a21edcc9e9ce-us-east-2.db.astra.datastax.com:29042:75b303bc-3b8d-4f1c-887c-0b1c312baa70> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


Closing connection <LibevConnection(139793671425568) aac415f8-3179-4fd1-bed8-a21edcc9e9ce-us-east-2.db.astra.datastax.com:29042:75b303bc-3b8d-4f1c-887c-0b1c312baa70> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


Downgrading core protocol version from 5 to 4 for aac415f8-3179-4fd1-bed8-a21edcc9e9ce-us-east-2.db.astra.datastax.com:29042:75b303bc-3b8d-4f1c-887c-0b1c312baa70. To avoid this, it is best practice to explicitly set Cluster(protocol_version) to the version supported by your cluster. http://datastax.github.io/python-driver/api/cassandra/cluster.html#cassandra.cluster.Cluster.protocol_version


Found multiple hosts with the same endpoint (aac415f8-3179-4fd1-bed8-a21edcc9e9ce-us-east-2.db.astra.datastax.com:29042:75b303bc-3b8d-4f1c-887c-0b1c312baa70). Excluding peer 10.5.212.32


In [ ]:
# To write data in db DO NOT RUN
df = pd.read_json("data.json")
print(df.info())
result_dict = df.to_dict(orient='records')
for item in result_dict:
    item['content'] = item['keywords'].upper() + ": \n" +item['content']
documents = []
for item in result_dict:
    for key, value in item.items():
        if isinstance(value, str):  # Check if the value is a string
            item[key] = value.strip()  # Strip whitespace
for text in result_dict:
    documents.append(Document(id_=text["url"],text=text["content"]))
print(len(documents))
index = VectorStoreIndex.from_documents(
    documents = documents,
    storage_context= storage_context,
    vector_store=cassandra_store,
    llm = llm,
    embed_model = embed_model,
    callback_manager=callback_manager,
    show_progress=True
)

In [ ]:
index = VectorStoreIndex.from_vector_store(
        storage_context=storage_context,
        vector_store=cassandra_store,
        llm = llm,
        embed_model = embed_model,
    )

**********
Trace: index_construction
**********


In [ ]:
import re

def remove_special_characters_except_dot_and_question(input_string):
    pattern = r'^[^a-zA-Z0-9?.]+|[^a-zA-Z0-9?.]+$'
    modified_string = re.sub(pattern, '', input_string)

    return modified_string

In [ ]:
def prompt_condense(questions):
    questions = list(map(str.strip, questions))
    # questions = questions[::-1]
    # questions = ["{}. {}".format(index, item) for index, item in enumerate(questions, start=1)]
    SYSTEM_PROMPT_CONDENSE = """
If the question is a greeting or sign off, whether formal or casual, DO NOT modify the meaning of the original question and let it be a greeting or sign off.

List of abbreviations:

    SVCE - Sri Venkateswara College of Engineering
    HOD - Head of Department
    AHoD - Assistant Head of Department
    AI - Artificial Intelligence
    IT - Information Technology
    INT - Information Technology
    CS - Computer Science and Engineering
    CSE - Computer Science and Engineering
    CPO - Chief Placement Officer
    APO - Assistant Placement Officer
    PO - Program Outcome
    PSO - Program Specific Outcome
    PEO - Program Educational Outcome
    dept - department

If there is no abbreviation to expand return the exact same question without modification.


Expand the abbreviation in the question and put the contracted abbreviation in parentheses right after the expansion. Stick to the list of abbreviations.


Output only the question and not any additional explanation or information.
Do not return the entire prompt but only the question.
And no side Notes
And Do not answer the question.
"""
    # print(SYSTEM_PROMPT_CONDENSE)
    return (SYSTEM_PROMPT_CONDENSE)

In [ ]:
condenseLLM = gradient.get_base_model(base_model_slug="llama2-70b-chat",)

In [ ]:
class LLM_run():
    def __init__(self):
        self.questions = []
        self.chat_memory_buff = ChatMemoryBuffer.from_defaults(llm=llm,token_limit=8192)
        self.question = ""
        self.question_con = ""
        self.nodes = []
        self.context_str = ""
        self.custom_chat_history = [
            ChatMessage(
                role=MessageRole.USER,
                content="Hello SVCE AI, I would like to have an informative chat about Sri Venkateswara College of Engineering (SVCE).",
            ),
            ChatMessage(role=MessageRole.ASSISTANT, content="Okay, sounds good."),
        ]
        self.chat_engine = None
    def make_condense(self):
        self.questions = []
        for chat in self.custom_chat_history:
            if chat.role == MessageRole.USER:
                self.questions.append(f"{chat.content}")
    def get_question(self,question):
        self.question = question
    def condense_prompt(self):
        SYSTEM_PROMPT_CONDENSE = prompt_condense(self.questions)
        # message = ChatMessage(role="user", content = SYSTEM_PROMPT_CONDENSE + "\n\n\nGiven Question: " + self.question)
        # condense_response = condenseLLM.chat([message])
        condense_response = condenseLLM.complete(query=prompt_condense([]) + ' \n\n Given Question: \n\n\n'+ self.question)
        self.question_con = str(condense_response.generated_output).strip()
        self.question_con = self.question_con.replace("Condense", "")
        self.question_con = self.question_con.replace("Question", "")
        self.question_con = self.question_con.replace("condense", "")
        self.question_con = self.question_con.replace("question", "")
        self.question_con = self.question_con.replace("assistant:", "").strip()
        # self.question_con = remove_special_characters_except_dot_and_question(self.question_con).strip()
        # self.question_con = self.question
        print("Condense Question : ", self.question_con)
    def reset(self):
        self.questions = []
        self.question = ""
        self.question_con = ""
        self.nodes = []
        self.context_str = ""
        self.chat_engine.reset()
        self.chat_memory_buff = ChatMemoryBuffer.from_defaults(llm=llm,token_limit=8192)
    def retrive_nodes(self):
        retriever = index.as_retriever(similarity_top_k=5)
        nodes1 = retriever.retrieve(self.question_con)
        nodes2 = retriever.retrieve(self.question)
        self.nodes = []
        self.nodes.extend(nodes1)
        self.nodes.extend(nodes2)
        str_node = set()
        for node in self.nodes:
            str_node.add(node.text.strip())
        self.context_str = ("\n\n".join(list(str_node)))
        # print("Context Nodes : \n", (self.context_str))
    def chatter(self,question):
        # self.make_condense()
        self.get_question(question)
        self.condense_prompt()
        # self.retrive_nodes()
        self.chat_engine = index.as_chat_engine(
                llm=llm,
                verbose=True,
                memory=self.chat_memory_buff,
                chat_mode="context",
                system_prompt=(
                    "You are a chatbot for Sri Venkateswara College of Engineering (SVCE) called SVCE AI developed by students of SVCE, able to have normal interactions, as well as talk"
                    "Please stick on to the given context and think twice before you answer any question. "
                ),
                context_template=(
                    "You are a chatbot for Sri Venkateswara College of Engineering (SVCE) called SVCE AI developed by students of SVCE, able to have normal interactions, as well as talk"
                    "Please stick on to the given context and think twice before you answer any question. "
                    "\n\nDon't blabber wrong answer outside the context. "

                    "Here are the relevant documents for the context:\n"
                    "{context_str}"

                    "\nInstruction: Use the previous chat history, or the context above, to interact and help the user."
                    "DO NOT HALLUCINATE AT ANY POINT OF TIME. STICK TO THE CONTEXT AS MUCH AS POSSIBLE "
                    "Always start answering by saying SVCE AI: "
                )
        )
        answer = self.chat_engine.stream_chat(self.question_con)
        # print(answer)
        self.custom_chat_history.extend([ChatMessage(
                role=MessageRole.USER,
                content = self.question_con,
            ),
            ChatMessage(role=MessageRole.ASSISTANT, content=str(answer))
        ])

        return answer


In [ ]:
chat = LLM_run()

In [ ]:
chat.chatter("who is the HOD of IT")

Condense Question :  who is the Head of Department of Information Technology (HOD of IT)


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: chat
    |_retrieve -> 0.491932 seconds
      |_embedding -> 0.388646 seconds
    |_llm -> 0.0 seconds
**********


StreamingAgentChatResponse(response='', sources=[ToolOutput(content="system: You are a chatbot for Sri Venkateswara College of Engineering (SVCE) called SVCE AI developed by students of SVCE, able to have normal interactions, as well as talkPlease stick on to the given context and think twice before you answer any question.\nYou are a chatbot for Sri Venkateswara College of Engineering (SVCE) called SVCE AI developed by students of SVCE, able to have normal interactions, as well as talkPlease stick on to the given context and think twice before you answer any question. \n\nDon't blabber wrong answer outside the context. Here are the relevant documents for the context:\nINFORMATION-TECHNOLOGY PEOPLE FACULTY: \n\nFaculty\n    Professors\n        Dr.  V. Vidhya,  M.E., Ph.D, Head of the Department\n        Dr.  G. Sumathi,  M.E., Ph.D\n        Dr.  V.M. Sivagami,  M.E., Ph.D\n        Dr.  C. Yaashuwanth,  M.E., Ph.D\n        Dr.  T. Sukumar,  M.Tech., Ph.D, Assistant HoD\n\n    Associate 

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import StreamingResponse
import time

import nest_asyncio
from pyngrok import ngrok
import uvicorn
from pydantic import BaseModel
!pyngrok authtoken 22UrU24nFcbiZxNmX5vCUja1XJ9_2ogXkzeaQ6kcjka9cdMDz

class Item(BaseModel):
    question: str

def generate_data(streaming_response):
    for text in streaming_response.response_gen:
        yield text

app = FastAPI()

origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post('/chat')
async def home(item: Item):
  return StreamingResponse(generate_data(chat.chatter(item.question.strip())))

@app.get('/reset')
async def reset():
    chat.reset()
    return {"message": "Done"}

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: https://561a-34-121-21-18.ngrok-free.app


INFO:     Started server process [630]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     136.226.243.26:0 - "OPTIONS /chat/ HTTP/1.1" 200 OK
INFO:     136.226.243.26:0 - "POST /chat/ HTTP/1.1" 307 Temporary Redirect
INFO:     136.226.243.26:0 - "OPTIONS /chat HTTP/1.1" 200 OK
Condense Question :  Hi


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: chat
    |_retrieve -> 0.415284 seconds
      |_embedding -> 0.320449 seconds
    |_llm -> 0.0 seconds
**********
INFO:     136.226.243.26:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     2409:4072:8e8a:c801:9134:e48b:80d1:5d20:0 - "OPTIONS /chat/ HTTP/1.1" 200 OK
INFO:     2409:4072:8e8a:c801:9134:e48b:80d1:5d20:0 - "POST /chat/ HTTP/1.1" 307 Temporary Redirect
INFO:     2409:4072:8e8a:c801:9134:e48b:80d1:5d20:0 - "OPTIONS /chat HTTP/1.1" 200 OK
Condense Question :  hi


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: chat
    |_retrieve -> 0.487819 seconds
      |_embedding -> 0.384368 seconds
    |_llm -> 0.0 seconds
**********
INFO:     2409:4072:8e8a:c801:9134:e48b:80d1:5d20:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     2409:4072:8e8a:c801:9134:e48b:80d1:5d20:0 - "POST /chat/ HTTP/1.1" 307 Temporary Redirect
Condense Question :  write a code for n queens problem


...

Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: chat
    |_retrieve -> 0.412635 seconds
      |_embedding -> 0.310877 seconds
    |_llm -> 0.0 seconds
**********
INFO:     2409:4072:8e8a:c801:9134:e48b:80d1:5d20:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     2409:4072:8e8a:c801:9134:e48b:80d1:5d20:0 - "POST /chat/ HTTP/1.1" 307 Temporary Redirect
Condense Question :  tell something about me


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: chat
    |_retrieve -> 0.435165 seconds
      |_embedding -> 0.333495 seconds
    |_llm -> 0.0 seconds
**********
INFO:     2409:4072:8e8a:c801:9134:e48b:80d1:5d20:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     2409:4072:8e8a:c801:9134:e48b:80d1:5d20:0 - "POST /chat/ HTTP/1.1" 307 Temporary Redirect
Condense Question :  i never said that


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: chat
    |_retrieve -> 0.425403 seconds
      |_embedding -> 0.323079 seconds
    |_llm -> 0.0 seconds
**********
INFO:     2409:4072:8e8a:c801:9134:e48b:80d1:5d20:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     2409:4072:8e8a:c801:9134:e48b:80d1:5d20:0 - "POST /chat/ HTTP/1.1" 307 Temporary Redirect
Condense Question :  no problem


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: chat
    |_retrieve -> 0.519132 seconds
      |_embedding -> 0.418932 seconds
    |_llm -> 0.0 seconds
**********
INFO:     2409:4072:8e8a:c801:9134:e48b:80d1:5d20:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     2409:4072:8e8a:c801:9134:e48b:80d1:5d20:0 - "POST /chat/ HTTP/1.1" 307 Temporary Redirect
Condense Question :  who is available in room number 11-104


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: chat
    |_retrieve -> 0.50233 seconds
      |_embedding -> 0.396255 seconds
    |_llm -> 0.0 seconds
**********
INFO:     2409:4072:8e8a:c801:9134:e48b:80d1:5d20:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     2409:4072:8e8a:c801:9134:e48b:80d1:5d20:0 - "GET /reset/ HTTP/1.1" 307 Temporary Redirect
INFO:     2409:4072:8e8a:c801:9134:e48b:80d1:5d20:0 - "GET /reset HTTP/1.1" 200 OK


In [ ]:
chat.chatter("Who are the HODs of CS and IT?")

In [ ]:
chat.chatter("How many Faculties are there in CSE and IT?")

**********
Trace: chat
    |_llm ->  1.709317 seconds
**********
Condense Question :  What is the total number of faculty members in the Computer Science and Engineering (CSE) and Information Technology (IT) departments at SVCE?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.479085 seconds
      |_embedding ->  0.35626 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.499232 seconds
      |_embedding ->  0.377683 seconds
**********
Context Nodes : 
 INFORMATION-TECHNOLOGY FACILITIES DEPT-LIBRARY: 

Department Library
    Library is a warehouse of knowledge encapsulated in the form of books which can bring a major transformation to the society if it falls in the right pair of hands. SVCE's Department of Information Technology has its very own library situated on the second floor of the block. The library in the department has a wide collection of technical books enabling easy access for both the faculty as well as students.

Library Book details
    S.No._Particulars_Total
    1_Total Number of Volumes_1715
    2_Total Number of Titles_984
    3_Total Number of International Journals_139
    4_Total Number of Nationa

In [ ]:
chat.chatter("How is placements in SVCE?")

**********
Trace: chat
    |_llm ->  1.575286 seconds
**********
Condense Question :  What is the placement scenario at Sri Venkateswara College of Engineering (SVCE)?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.436664 seconds
      |_embedding ->  0.338084 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.398001 seconds
      |_embedding ->  0.274408 seconds
**********
Context Nodes : 
 PLACEMENT FEEDBACK RECRUITERS: 

Recruiters
    Thanks for all the great support you have provided to the recruiting process. The candidates were enthusiastic and enhanced by core elements. I'm looking forward to joining us.
    -Mr A Ravindra Babu, Director, EY

    The students are very involved and highly professional. Next year we would be happy to visit the campus.
    -Ms Arati Mohanram, AVP and India HR Head, Mr.Cooper Group

    The students were happy to be included in our founding party. The team is filled with resources, experience and ideas. We look forward to a synergical partnership in future and recruit many more of you for our other ventures from your res

In [ ]:
chat.chatter("How to get admission in SVCE?")

**********
Trace: chat
    |_llm ->  1.627321 seconds
**********
Condense Question :  What are the requirements and procedures for admission into SVCE's CSE and IT departments?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.408247 seconds
      |_embedding ->  0.28389 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.45401 seconds
      |_embedding ->  0.325767 seconds
**********
Context Nodes : 
 ADMISSION ADMISSION-PROCEDURE ME-MTECH-ADMISSIONS: 

    M.E./ M.Tech Admissions
        Admissions to all M.E./M.Tech programs at SVCE will be based on marks or grades obtained in B.E./B.Tech program and scores of TANCET/CET conducted by Consortium of Colleges as per Tamilnadu Common Entrance Test Regulations.

    Programs Offered
        1. M.E. Industrial automation and Robotics
        2. M.E. Mechatronics
        3. M.E. Communication Systems
        4. M.E. Power Electronics & Drives
        5. M.E. Computer Science & Engineering
        6. M.E. Computer Science & Engineering ( Networks )
        7. M.Tech. Chemical Engineering
        8. M.Tech. Bio Technology

  

In [ ]:
chat.chatter("Who to contact for admission")

**********
Trace: chat
    |_llm ->  1.56496 seconds
**********
Condense Question :  Condense Question: Who should be contacted for admission inquiries at SVCE?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.468592 seconds
      |_embedding ->  0.345285 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.37967 seconds
      |_embedding ->  0.257489 seconds
**********
Context Nodes : 
 ADMISSION ADMISSION-PROCEDURE ME-MTECH-ADMISSIONS: 

    M.E./ M.Tech Admissions
        Admissions to all M.E./M.Tech programs at SVCE will be based on marks or grades obtained in B.E./B.Tech program and scores of TANCET/CET conducted by Consortium of Colleges as per Tamilnadu Common Entrance Test Regulations.

    Programs Offered
        1. M.E. Industrial automation and Robotics
        2. M.E. Mechatronics
        3. M.E. Communication Systems
        4. M.E. Power Electronics & Drives
        5. M.E. Computer Science & Engineering
        6. M.E. Computer Science & Engineering ( Networks )
        7. M.Tech. Chemical Engineering
        8. M.Tech. Bio Technology

  

In [ ]:
chat.chatter("Admission in B.Tech?")

**********
Trace: chat
    |_llm ->  1.644518 seconds
**********
Condense Question :  What are the requirements and procedures for admission into SVCE's CSE and IT departments?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.405917 seconds
      |_embedding ->  0.27356 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.459496 seconds
      |_embedding ->  0.335563 seconds
**********
Context Nodes : 
 ADMISSION ADMISSION-PROCEDURE ME-MTECH-ADMISSIONS: 

    M.E./ M.Tech Admissions
        Admissions to all M.E./M.Tech programs at SVCE will be based on marks or grades obtained in B.E./B.Tech program and scores of TANCET/CET conducted by Consortium of Colleges as per Tamilnadu Common Entrance Test Regulations.

    Programs Offered
        1. M.E. Industrial automation and Robotics
        2. M.E. Mechatronics
        3. M.E. Communication Systems
        4. M.E. Power Electronics & Drives
        5. M.E. Computer Science & Engineering
        6. M.E. Computer Science & Engineering ( Networks )
        7. M.Tech. Chemical Engineering
        8. M.Tech. Bio Technology

 

In [ ]:
chat.chatter("List all the faculties with name in CSE and IT dept")

**********
Trace: chat
    |_llm ->  1.779928 seconds
**********
Condense Question :  Condense Question: Can you provide a list of faculty members in both CSE and IT departments at SVCE?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.447149 seconds
      |_embedding ->  0.321286 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.443887 seconds
      |_embedding ->  0.318151 seconds
**********
Context Nodes : 
 COMPUTER-SCIENCE-AND-ENGINEERING FACILITIES DEPT-LIBRARY: 

Dept. Library
    The Department Library of Computer Science and Engineering, gives a great resources and good ambience to the faculty,staff and students to enrich their knowledge. The Department Library has 1674 volumes to keep student and faculty abreasts of developments in research. The library resources are constantly being expanded by adding some amount of books in each semester. The library is air conditioned, spacious enough and designed to be a quiet environment for the readers. Library open from 8.30am to 4.30pm.

INFORMATION-TECHNOLOGY PEOPLE STAFF: 

Instructors
    Mr.  L. Francis Inigo Raj 
    Mr. 

In [ ]:
chat.chatter("How many are there?")

**********
Trace: chat
    |_llm ->  1.607675 seconds
**********
Condense Question :  How many HODs are there for CSE and IT departments at SVCE?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.609306 seconds
      |_embedding ->  0.484409 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.438261 seconds
      |_embedding ->  0.314594 seconds
**********
Context Nodes : 
 INFORMATION-TECHNOLOGY PEOPLE STAFF: 

Instructors
    Mr.  L. Francis Inigo Raj 
    Mr.  S. Sundarnarayanan 
    Mr.  K. Mohan 
    Ms. A. Gajalakshmi M.C.A.
    Mr.  M. Eliyas Ahamed 

Computer Operator
    Mr.  S. Manigandan 

Junior Assistant
    Ms.  J. Ambica 

Office Assistant
    Mr.  V. Naveen Kumar

INFORMATION-TECHNOLOGY PEOPLE FACULTY: 

Faculty
    Professors
        Dr.  V. Vidhya,  M.E., Ph.D, Head of the Department
        Dr.  G. Sumathi,  M.E., Ph.D
        Dr.  V.M. Sivagami,  M.E., Ph.D
        Dr.  C. Yaashuwanth,  M.E., Ph.D
        Dr.  T. Sukumar,  M.Tech., Ph.D, Assistant HoD

    Associate Professors
        Dr. N. Gobalakrishnan, M.Tech., Ph. 

In [ ]:
chat.chatter("Who is P. Leela rani")

**********
Trace: chat
    |_llm ->  1.867379 seconds
**********
Condense Question :  Condense Question: Who is P. Leela Rani in relation to SVCE?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.44163 seconds
      |_embedding ->  0.348246 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.430611 seconds
      |_embedding ->  0.330766 seconds
**********
Context Nodes : 
 PROFILE INFORMATION-TECHNOLOGY ELAKYA: 

Name	Mrs. R. Elakya
Designation	Assistant Professor
Areas of Interest	Deep Learning, Image Processing
Room No.	11-205
Intercom No.	361
Mobile	9884852306
Email Id (Official)	elakyar@svce.ac.in
Email Id (Personal)	elakyaragesh@gmail.com

PLACEMENT PLACEMENT-CELL: 

How Placements Cell Operates in SVCE
    The Placement cell is one of the important department in SVCE. Knowing that you will be guided after your graduation so that you land a job helps you feel secure. This is the reason why we have a dedicated placement cell that helps, guides, and advises students about the future.
    In SVCE, students come from all walks of lives. T

In [ ]:
chat.chatter("Who is thiyagarajan")

**********
Trace: chat
    |_llm ->  1.855903 seconds
**********
Condense Question :  What is the role of Thiyagarajan at SVCE?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.416782 seconds
      |_embedding ->  0.294331 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.398959 seconds
      |_embedding ->  0.27555 seconds
**********
Context Nodes : 
 PROFILE INFORMATION-TECHNOLOGY ELAKYA: 

Name	Mrs. R. Elakya
Designation	Assistant Professor
Areas of Interest	Deep Learning, Image Processing
Room No.	11-205
Intercom No.	361
Mobile	9884852306
Email Id (Official)	elakyar@svce.ac.in
Email Id (Personal)	elakyaragesh@gmail.com

PLACEMENT PLACEMENT-CELL: 

How Placements Cell Operates in SVCE
    The Placement cell is one of the important department in SVCE. Knowing that you will be guided after your graduation so that you land a job helps you feel secure. This is the reason why we have a dedicated placement cell that helps, guides, and advises students about the future.
    In SVCE, students come from all walks of lives. Th

In [ ]:
chat.chatter("Who is sukumar?")

**********
Trace: chat
    |_llm ->  1.916535 seconds
**********
Condense Question :  Condense Question: Who is Sukumar related to SVCE?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.39808 seconds
      |_embedding ->  0.271631 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.436942 seconds
      |_embedding ->  0.333421 seconds
**********
Context Nodes : 
 PROFILE COMPUTER-SCIENCE-AND-ENGINEERING DEEPARANI: 

Name	S. Deeparani
Designation	Assistant professor
Areas of Interest	Machine Learning	
Mobile	7540070696
Email Id (Official)	deeparanis@svce.ac.in
Email Id (Personal)	sdeeparanisenthil2@gmail.com

PROFILE COMPUTER-SCIENCE-AND-ENGINEERING SENTHIL-KUMAR: 

Name	Mr. R. Senthil Kumar
Designation	Assistant Professor
Areas of Interest	Computer Networks,Cloud Computing	
Intercom No.	311
Email Id (Official)	senthilkumar_r@svce.ac.in

PLACEMENT PLACEMENT-CELL: 

How Placements Cell Operates in SVCE
    The Placement cell is one of the important department in SVCE. Knowing that you will be guided after your graduation so that yo

In [ ]:
chat.chatter("Who found SVCE IT department and When?")

**********
Trace: chat
    |_llm ->  1.992098 seconds
**********
Condense Question :  Condense Question: Who established the IT department at SVCE and when was it founded?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.506635 seconds
      |_embedding ->  0.384512 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.466744 seconds
      |_embedding ->  0.340391 seconds
**********
Context Nodes : 
 COMPUTER-SCIENCE-AND-ENGINEERING TECHNICAL-ASSOCIATIONS ACM: 

ACM
    Association of Computing Machinery, Student Chapter
        SVCE ACM Student Chapter was set up by students of CSE department under the guidance of ACM Student Chapter faculty coordinator. It was officially chartered by ACM New York on 26th August 2011. ACM is an international organization whose aim is "Advancing computing as a science and profession". A team of 26 enthusiastic students from the Department took the initiative to set up this chapter. The official launch was on 22nd September 2011. SVCE ACM student chapter was set up to facilitate all activities which aim at spreading quality computing 

In [ ]:
chat.chatter("Hey Hi")

**********
Trace: chat
    |_llm ->  3.908021 seconds
**********
Condense Question :  What is the name of the Head of Department for Information Technology at SVCE?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.446543 seconds
      |_embedding ->  0.31721 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.480826 seconds
      |_embedding ->  0.352074 seconds
**********
Context Nodes : 
 INFORMATION-TECHNOLOGY PLACEMENTS: 

Placements
    Placement Coordinators
        Mr. AR. Guru Gokul, Assistant Professor, Assistant Placement Officer
        Mr. N. Selvaganesh, Assistant Professor

OUR RECRUITERS
    Amazon, Mr. Cooper, EmbedUR, EY, Freshworks, Informatica, LatentView, Altimetrik, Mu Sigma, Temenos, Accenture, Zoho

PROFILE INFORMATION-TECHNOLOGY VIDHYA: 

Name	Dr. V. Vidhya, ME, Ph.D.
Designation	Professor & Head of the Department
Areas of Interest	Natural Language Processing, Speech Coding, Machine Learning, Data Mining
Room No.	Department of IT,First Floor Post Box No: 3, Pennalur, Sriperumbudur-602117
Intercom No.	Ph:27152000 Ext:350
Email Id (Off

In [ ]:
chat.chatter("How to contact via phone HOD IT")

**********
Trace: chat
    |_llm ->  2.144872 seconds
**********
Condense Question :  Condense Question: What is the contact number of HOD IT at SVCE?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.410206 seconds
      |_embedding ->  0.313152 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.366635 seconds
      |_embedding ->  0.267617 seconds
**********
Context Nodes : 
 PROFILE INFORMATION-TECHNOLOGY VIDHYA: 

Name	Dr. V. Vidhya, ME, Ph.D.
Designation	Professor & Head of the Department
Areas of Interest	Natural Language Processing, Speech Coding, Machine Learning, Data Mining
Room No.	Department of IT,First Floor Post Box No: 3, Pennalur, Sriperumbudur-602117
Intercom No.	Ph:27152000 Ext:350
Email Id (Official)	hodit@svce.ac.in
Email Id (Personal)	vidhyav@svce.ac.in

INFORMATION-TECHNOLOGY PEOPLE STAFF: 

Instructors
    Mr.  L. Francis Inigo Raj 
    Mr.  S. Sundarnarayanan 
    Mr.  K. Mohan 
    Ms. A. Gajalakshmi M.C.A.
    Mr.  M. Eliyas Ahamed 

Computer Operator
    Mr.  S. Manigandan 

Junior Assistant
    Ms.  J. Ambica 

Offic

In [ ]:
chat.chatter("How to contact CS HOD")

**********
Trace: chat
    |_llm ->  2.291464 seconds
**********
Condense Question :  Condense Question: Who should be contacted for CSE HOD in SVCE?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.558714 seconds
      |_embedding ->  0.458112 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.346685 seconds
      |_embedding ->  0.251237 seconds
**********
Context Nodes : 
 PROFILE COMPUTER-SCIENCE-AND-ENGINEERING GNANASEKAR: 

Name	Dr. J.M.Gnanasekar
Designation	Professor
Areas of Interest	Web Technology,Cloud Computing,Service Oriented Computing
Room No.	5106
Intercom No.	304
Mobile	9444508360
Email Id (Official)	jmg@svce.ac.in

PROFILE COMPUTER-SCIENCE-AND-ENGINEERING JANANI: 

Name	Ms.S.Janani
Designation	Assistant Professor
Areas of Interest	Networks,Object Oriented Programming,Object Oriented Analysis and Design
Room No.	5011
Intercom No.	301
Mobile	9677741326
Email Id (Official)	sjanani@svce.ac.in
Email Id (Personal)	jananisivapat@gmail.com

PROFILE COMPUTER-SCIENCE-AND-ENGINEERING KIRUTHIGA: 

Name	Ms.E.Kiruthiga
Designation	Assista

In [ ]:
chat.chatter("How to contact Leela rani")

**********
Trace: chat
    |_llm ->  2.354457 seconds
**********
Condense Question :  Condense Question: What is the contact information for P. Leela Rani at SVCE?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.503434 seconds
      |_embedding ->  0.382188 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.464924 seconds
      |_embedding ->  0.365268 seconds
**********
Context Nodes : 
 PROFILE INFORMATION-TECHNOLOGY ELAKYA: 

Name	Mrs. R. Elakya
Designation	Assistant Professor
Areas of Interest	Deep Learning, Image Processing
Room No.	11-205
Intercom No.	361
Mobile	9884852306
Email Id (Official)	elakyar@svce.ac.in
Email Id (Personal)	elakyaragesh@gmail.com

PROFILE COMPUTER-SCIENCE-AND-ENGINEERING POORANI: 

Name	Ms. S Poorani
Designation	Assistant Professor
Areas of Interest	Wireless Networks,Information Security
Room No.	5010
Intercom No.	312
Mobile	9940640691
Email Id (Official)	spoorani@svce.ac.in
Email Id (Personal)	poorani703@gmail.com

PLACEMENT PLACEMENT-CELL: 

How Placements Cell Operates in SVCE
    The Placement cell is one of the important

In [ ]:
chat.chatter("You are great thnx")

**********
Trace: chat
    |_llm ->  2.31451 seconds
**********
Condense Question :  Condense Question: Thank you for your assistance.


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.502427 seconds
      |_embedding ->  0.376948 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.444515 seconds
      |_embedding ->  0.315963 seconds
**********
Context Nodes : 
 PROFILE INFORMATION-TECHNOLOGY SUGACINI: 

Name	M.Sugacini , M.E.
Designation	Assistant Professor
Areas of Interest	Wireless Sensor Networks, Operating Systems, DBMS
Room No.	Room No: 11-204
Intercom No.	Extn:361	
Email Id (Official)	msugacini@svce.ac.in
Email Id (Personal)	msugacini@gmail.com

PROFILE COMPUTER-SCIENCE-AND-ENGINEERING DEEPARANI: 

Name	S. Deeparani
Designation	Assistant professor
Areas of Interest	Machine Learning	
Mobile	7540070696
Email Id (Official)	deeparanis@svce.ac.in
Email Id (Personal)	sdeeparanisenthil2@gmail.com

PLACEMENT FEEDBACK: 

Feedback
    Thanks for all the great support you have provided to the recruiting process. The candidates were 

In [ ]:
chat.chatter("Cuttoffs?")

**********
Trace: chat
    |_llm ->  2.731724 seconds
**********
Condense Question :  Condense Question: What are the cutoff marks for admission into CSE and IT departments at SVCE?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.532698 seconds
      |_embedding ->  0.407132 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.431027 seconds
      |_embedding ->  0.333931 seconds
**********
Context Nodes : 
 PROFILE COMPUTER-SCIENCE-AND-ENGINEERING DEEPARANI: 

Name	S. Deeparani
Designation	Assistant professor
Areas of Interest	Machine Learning	
Mobile	7540070696
Email Id (Official)	deeparanis@svce.ac.in
Email Id (Personal)	sdeeparanisenthil2@gmail.com

PLACEMENT PLACEMENT-STATISTICS: 

Highest CTC Graph of our Students. (Batch 2021 - 22)
    Your greatest accomplishment is our happiness.
        Atlassian
        Amazon
        PayPal

ADMISSION ADMISSION-PROCEDURE ME-MTECH-ADMISSIONS: 

    M.E./ M.Tech Admissions
        Admissions to all M.E./M.Tech programs at SVCE will be based on marks or grades obtained in B.E./B.Tech program and scores of TANCET/CET conducted by Con

In [ ]:
chat.chatter("Annual Average CTC?")

**********
Trace: chat
    |_llm ->  3.144722 seconds
**********
Condense Question :  Condense Question: What is the annual average cost-to-company (CTC) for students placed in CSE and IT departments at SVCE?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.480325 seconds
      |_embedding ->  0.35156 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  0.397519 seconds
      |_embedding ->  0.270831 seconds
**********
Context Nodes : 
 PLACEMENT PLACEMENT-STATISTICS: 

Highest CTC Graph of our Students. (Batch 2021 - 22)
    Your greatest accomplishment is our happiness.
        Atlassian
        Amazon
        PayPal

PLACEMENT PLACEMENT-CELL: 

How Placements Cell Operates in SVCE
    The Placement cell is one of the important department in SVCE. Knowing that you will be guided after your graduation so that you land a job helps you feel secure. This is the reason why we have a dedicated placement cell that helps, guides, and advises students about the future.
    In SVCE, students come from all walks of lives. They come from different parts of the country including rural areas. Some individuals may n

In [ ]:
chat.chatter("What are the PO of B.Tech IT department?")

**********
Trace: chat
    |_llm ->  1.63705 seconds
**********
Condense Question :  What is the Program Outcome for the B.Tech IT department at SVCE?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  1.379132 seconds
      |_embedding ->  0.625402 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  1.169451 seconds
      |_embedding ->  0.415772 seconds
**********
Context Nodes : 
 INFORMATION-TECHNOLOGY DEPARTMENTS: 

Profile
    In 1996, Sri Venkateswara College of Engineering pioneered the introduction of the B.Tech degree programme in Information Technology under the affiliation of University of Madras. This is the first of its kind in Indian Universities. The department fulfills the requirements for the award of B.Tech Degree of Anna University.
    The Venture was initiated under the guidance of our patron Dr.A.C.Muthiah and with the blessings of the Kanchi Paramacharya. When we started, the batch size was 30. This increased to 60 in 1998 and scaled up to 120 by the year 2000. Our growing numbers is indicative of the highly qualified and extr

In [ ]:
chat.chatter("What are the PSO of B.Tech IT department?")

**********
Trace: chat
    |_llm ->  2.202517 seconds
**********
Condense Question :  What are the Program Specific Outcomes (PSO) for the B.Tech Information Technology (IT) department at Sri Venkateswara College of Engineering (SVCE)?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  1.18123 seconds
      |_embedding ->  0.432333 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  1.269884 seconds
      |_embedding ->  0.521228 seconds
**********
Context Nodes : 
 COMPUTER-SCIENCE-AND-ENGINEERING PEOPLE FACULTY: 

Faculty
    Professors
        Dr. R. Anitha,  M.E., Ph.D,  Head of the Department
        Dr. J. M. Gnanasekar, M.E., Ph.D,   AHoD
        Dr. P. Janarthanan, M.E., Ph.D   
        Dr. R. Jayabhaduri, M.E., Ph.D   
        Dr. N. M. Balamurugan, M.E., Ph.D   

    Associate Professors
        Dr. N. Rajganesh, M.E., Ph.D   
        Dr. N. Revathi M.E., Ph.D
        Dr. P. Vinothiyalakshmi, M.E., Ph.D
        Dr. T. Rajasekaran, M.E., Ph.D
        Dr. V. Rajalakshmi, M.E., Ph.D
        Dr. T. Padmavathy M.E., Ph.D   
        Dr. S. Rajalakshmi, M.E.,Ph.D   
        Mr. M. Suresh Kumar, M.E.,(Ph.D),  

    Assistant Profe

In [ ]:
chat.chatter("What are the PSO of B.E CS department?")

**********
Trace: chat
    |_llm ->  1.892944 seconds
**********
Condense Question :  What are the Program Specific Outcomes for the B.Tech Computer Science and Engineering department at SVCE?


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  1.166035 seconds
      |_embedding ->  0.413454 seconds
**********


Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  1.172525 seconds
      |_embedding ->  0.423913 seconds
**********
Context Nodes : 
 COMPUTER-SCIENCE-AND-ENGINEERING PEOPLE FACULTY: 

Faculty
    Professors
        Dr. R. Anitha,  M.E., Ph.D,  Head of the Department
        Dr. J. M. Gnanasekar, M.E., Ph.D,   AHoD
        Dr. P. Janarthanan, M.E., Ph.D   
        Dr. R. Jayabhaduri, M.E., Ph.D   
        Dr. N. M. Balamurugan, M.E., Ph.D   

    Associate Professors
        Dr. N. Rajganesh, M.E., Ph.D   
        Dr. N. Revathi M.E., Ph.D
        Dr. P. Vinothiyalakshmi, M.E., Ph.D
        Dr. T. Rajasekaran, M.E., Ph.D
        Dr. V. Rajalakshmi, M.E., Ph.D
        Dr. T. Padmavathy M.E., Ph.D   
        Dr. S. Rajalakshmi, M.E.,Ph.D   
        Mr. M. Suresh Kumar, M.E.,(Ph.D),  

    Assistant Profe

# **Dont Run**



In [ ]:
questions = []
for chat in custom_chat_history:
    if chat.role == MessageRole.USER:
        questions.append(f"Human: {chat.content}")

In [ ]:
question = "What is Association of Information Technologists.?"

In [ ]:
SYSTEM_PROMPT_CONDENSE = """
You are an assistant for Sri Venkateswara College of Engineering (SVCE) given the below conversation create a condense question for the given query.
Expand the abbreviation with regard to a college in the question for sure, if any. Stick to the given list of abbreviations.

List of abbreviations:
    SVCE - Sri Venkateswara College of Engineering
    HOD - Head of Department
    IT - Information Technology
    CSE - Computer Science and Engineering
    CPO - Chief Placement Officer
    APO - Assistant Placement Officer
    PO - Program Outcome
    PSO - Program Specific Outcome
    PEO - Program Educational Outcome
    dept - department

Output only the question not any additional explanation.
And Do not answer the question
Conversation:
    {}


""".format("\n".join(questions))

In [ ]:
SYSTEM_PROMPT_CONDENSE

'\nYou are an assistant for Sri Venkateswara College of Engineering (SVCE) given the below conversation create a condense question for the given query.\nExpand the abbreviation with regard to a college in the question for sure, if any. Stick to the given list of abbreviations.\n\nList of abbreviations:\n    SVCE - Sri Venkateswara College of Engineering\n    HOD - Head of Department\n    IT - Information Technology\n    CSE - Computer Science and Engineering\n    CPO - Chief Placement Officer\n    APO - Assistant Placement Officer\n    PO - Program Outcome\n    PSO - Program Specific Outcome\n    PEO - Program Educational Outcome\n    dept - department\n\nOutput only the question not any additional explanation.\nAnd Do not answer the question\nConversation:\n    Human: Hello assistant, we are having a insightful chat about Sri Venkateswara College of Engineering (SVCE).\n\n\n'

In [ ]:
condense_engine = SimpleChatEngine.from_defaults(
    llm=llm,
    service_context=service_context,
    system_prompt= SYSTEM_PROMPT_CONDENSE,
)

In [ ]:
condense_response = condense_engine.chat(question)

**********
Trace: chat
    |_llm ->  1.70462 seconds
**********


In [ ]:
question_con = condense_response.response.strip()

In [ ]:
question_con

'What is the role and purpose of the Association of Information Technologists in SVCE?'

In [ ]:
retriever=index.as_retriever(similarity_top_k=5,)

In [ ]:
nodes1 = retriever.retrieve(question_con)

Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  1.366405 seconds
      |_embedding ->  0.62058 seconds
**********


In [ ]:
nodes2 = retriever.retrieve(question)

Server warning: Top-K queries can only be run with consistency level ONE / LOCAL_ONE / NODE_LOCAL. Consistency level LOCAL_QUORUM was requested. Downgrading the consistency level to LOCAL_ONE.
**********
Trace: query
    |_retrieve ->  1.212204 seconds
      |_embedding ->  0.461009 seconds
**********


In [ ]:
nodes = []
nodes.extend(nodes1)
nodes.extend(nodes2)
print(nodes)

[NodeWithScore(node=TextNode(id_='7f73b86e-5c09-49f7-9d45-02db9f43fea9', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='https://www.svce.ac.in/departments/computer-science-and-engineering/?page=technical-associations&subMenu=cyber-hub', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='1443e40e12d584a0f8ab108a2b6150adfd19ee65856945d26a4dc8e777120f40'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='f10c8346-b2e3-4c70-adc4-9e0df6d4bdc4', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='62da08e6357d54a1f0599af0fc5204a515e985579c0d2cfc06a684b4f559794b'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='bfb60f15-3b5d-4fcf-8128-85e2d856baa0', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='dec3a78fe224dc6f6e7cd54e38fe2bb6db22f0c9afe222c061c1dc789465e2e1')}, text='COMPUTER-SCIENCE-AND-ENGINEERING TECHNICAL-ASSOCIATIONS CYBER-HUB: \n\nSVCE Cyb

In [ ]:
str_node = set()
for node in nodes:
    str_node.add(node.text.strip())

In [ ]:
context_str = ("\n\n".join(list(str_node)))

In [ ]:
print(context_str)

COMPUTER-SCIENCE-AND-ENGINEERING TECHNICAL-ASSOCIATIONS ACM: 

ACM
    Association of Computing Machinery, Student Chapter
        SVCE ACM Student Chapter was set up by students of CSE department under the guidance of ACM Student Chapter faculty coordinator. It was officially chartered by ACM New York on 26th August 2011. ACM is an international organization whose aim is "Advancing computing as a science and profession". A team of 26 enthusiastic students from the Department took the initiative to set up this chapter. The official launch was on 22nd September 2011. SVCE ACM student chapter was set up to facilitate all activities which aim at spreading quality computing education by setting up learning groups, organizing seminars, conducting workshops, facilitating contests and so on.

    ACM-SVCE STUDENTS CHAPTER
        The SVCE ACM student chapter aims to provide an opportunity for students from different walks of life to compete and tackle problems in an effective manner. As one o

In [ ]:
chat_engine = SimpleChatEngine.from_defaults(
                llm=llm,
                verbose=True,
                nodes=nodes,
                callback_manager=callback_manager,
                system_prompt=(
                    "You are a chatbot for Sri Venkateswara College of Engineering (SVCE), able to have normal interactions, as well as talk"
                    "Please stick on to the given context and think twice before you answer any question. "
                    "Here are the relevant documents for the context:\n\n"

                    f"{context_str}"

                    "\n\nDon't blabber wrong answer outside the context. "
                    "If a list is being returned, return the complete exhaustive list instead of saying excuses for not providing complete answers. "
                    "DO NOT HALLUCINATE AT ANY POINT OF TIME. STICK TO THE CONTEXT AS MUCH AS POSSIBLE "
                    "Always start answering by saying SVCE AI: "
                )
)

In [ ]:
answer = chat_engine.stream_chat(question_con)

**********
Trace: chat
    |_llm ->  0.0 seconds
**********


In [ ]:
answer.print_response_stream()

 SVCE AI: The Association of Information Technologists (AIT) in SVCE plays a significant role in fostering creativity, technical skills, and overall well-being among students pursuing careers in the field of information technology. AIT aims to support students who are interested in and committed to IT disciplines by arranging various technical events, providing resources for learning about these disciplines conceptually and practically, and organizing webinars, seminars, hands-on sessions, and other events for practical experience.

In [ ]:
str(answer)

'SVCE AI: The Association of Information Technologists (AIT) in SVCE plays a significant role in fostering creativity, technical skills, and overall well-being among students pursuing careers in the field of information technology. AIT aims to support students who are interested in and committed to IT disciplines by arranging various technical events, providing resources for learning about these disciplines conceptually and practically, and organizing webinars, seminars, hands-on sessions, and other events for practical experience.'

In [ ]:
print(answer)

 SVCE AI: The Association of Information Technologists (AIT) in SVCE plays a significant role in fostering creativity, technical skills, and overall well-being among students pursuing careers in the field of information technology. AIT aims to support students who are interested in and committed to IT disciplines by arranging various technical events, providing resources for learning about these disciplines conceptually and practically, and organizing webinars, seminars, hands-on sessions, and other events for practical experience.


In [ ]:
custom_chat_history.extend([ChatMessage(
        role=MessageRole.USER,
        content = question_con,
    ),
    ChatMessage(role=MessageRole.ASSISTANT, content=str(answer))])

In [ ]:
custom_chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content='Hello assistant, we are having a insightful chat about Sri Venkateswara College of Engineering (SVCE).', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Okay, sounds good.', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='Which faculty professors are associated with the Information Technology department at SVCE?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=' SVCE AI: The following faculty professors are associated with the Information Technology department at Sri Venkateswara College of Engineering (SVCE):\n\n1. Dr. V. Vidhya, M.E., Ph.D, Head of the Department\n2. Dr. G. Sumathi, M.E., Ph.D\n3. Dr. V.M. Sivagami, M.E., Ph.D\n4. Dr. C. Yaashuwanth, M.E., Ph.D\n5. Dr. T. Sukumar, M.Tech., Ph.D, Assistant HoD\n6. Dr. N. Gobalakrishnan, M.Tech., Ph. D\n7. Dr. N. Devi, M.E., Ph.D\n8. Dr. D. Jayanthi, M.E., Ph.D\n9. Dr. P. L